In [1]:
import numpy as np
import scipy.stats as stats
import pickle
import os
import sys

In [2]:
def get_features_one_window(d):
    t, accel, gyro = d[:, 0], d[:, 1:4], d[:, 4:7]
    
    accel_mean = np.mean(accel, axis=0)
    accel_var = np.var(accel, axis=0)
    accel_skew = stats.skew(accel, axis=0)
    accel_kurt = stats.kurtosis(accel, axis=0)
    accel_rms = np.mean(accel*accel, axis=0)
    
    gyro_mean = np.mean(gyro, axis=0)
    gyro_var = np.var(gyro, axis=0)
    gyro_skew = stats.skew(gyro, axis=0)
    gyro_kurt = stats.kurtosis(gyro, axis=0)
    gyro_rms = np.mean(gyro*gyro, axis=0)
        
    res = np.zeros((1, 30))
    res[0, 0:3] = accel_mean
    res[0, 3:6] = accel_var
    res[0, 6:9] = accel_skew
    res[0, 9:12] = accel_kurt
    res[0, 12:15] = accel_rms
    
    res[0, 15:18] = gyro_mean
    res[0, 18:21] = gyro_var
    res[0, 21:24] = gyro_skew
    res[0, 24:27] = gyro_kurt
    res[0, 27:30] = gyro_rms
    
    return res

In [3]:
def get_features_one_session(data, window_size=6*16, step_size=3*16):
    count = (len(data)-window_size)//step_size + 1
    res = np.zeros((count, 31))
    
    print("Data sahpe: ", data.shape, "Feature shape: ", res.shape)
    for i in range(count):
        si = i*step_size        
        d = data[si:si+window_size, :]
        res[i, 0] = si
        res[i, 1:] = get_features_one_window(d)
    return res

In [4]:
def get_features_steven_lab():
    path = 'C:/ASM/DevData/eating_steven' if "C:" in os.getcwd() else '.'        
        
    with open(path+'/data/lab_data_steven_smoothed.pkl', 'rb') as file:
        data = pickle.load(file)
    
    features = []    
    for subj in range(len(data)):
        fsubj =[]
        for sess in range(len(data[subj])):
            print("Extracing features... Subj, sess: ", subj, sess)
            d = data[subj][sess]["data_right"]
            f = get_features_one_session(d)
            print("Feature shape: ", f.shape)
            fsubj.append(f)
            
        features.append(fsubj)
        
    with open(path+'/features/lab_features_steven_right_gatech.pkl', 'wb') as file:
        pickle.dump(features, file)
        
    return features

In [5]:
def get_features_steven_free():
    path = 'C:/ASM/DevData/eating_steven' if "C:" in os.getcwd() else '.'    
    
    with open(path + '/data/free_data_steven_right_smoothed.pkl', 'rb') as file:
        data = pickle.load(file)
    
    features = []    
    for subj in range(len(data)):
        fsubj =[]
        for sess in range(len(data[subj])):
            print("Extracing features... Subj, sess: ", subj, sess)
            d = data[subj][sess]
            f = get_features_one_session(d)
            print("Feature shape: ", f.shape)
            fsubj.append(f)
            
        features.append(fsubj)
    
    with open(path+'/features/free_features_steven_right_gatech.pkl', 'wb') as file:
        pickle.dump(features, file)    
        
    return features

In [6]:
lab_free = sys.argv[1]
print("Data type: ", lab_free)

if lab_free=='lab':
    get_features_steven_lab() 
else:
    get_features_steven_free() 

Data type:  -f


FileNotFoundError: [Errno 2] No such file or directory: 'C:/ASM/DevData/eating_steven/data/free_data_steven_right.pkl'